<a href="https://colab.research.google.com/github/betoval/learning-tensorflow/blob/master/titanic_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [548]:
#we use pandas to load the csv files
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
#show some of the data using pandas
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


We note that we have the word "NaN" in the 'Age' feature, which means that we are missing those values.

In [549]:
total = train.isnull().sum().sort_values(ascending=False)
total.head()

Cabin       687
Age         177
Embarked      2
Fare          0
Ticket        0
dtype: int64

In [550]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

Here, we note that we are missing 177 values of Age, 687 of Cabin, and 2 of Embarked. As a first attemp we will replace the missing values of Age with the value of the "mean age" and the Embarked values with S, which is the most common.

In [551]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


We can see that the data has 11 features + Survived (the feature we are interested in).

Below, we can examine the statistics of the data using `pd.DataFrame.describe`

In [552]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


It is evident that we are missing 177 'Age' values. We need to take care of that.

Of course, not all features are useful. In fact, we don't need the following: PassengerID, ticket, Name because they don't tell us anything about the survival rate. It is important to note that the Cabin feature could be useful, however, we will drop it because of its missing values.

In [553]:
#show train columns
train.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [554]:
#show test columns
test.columns.values

array(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [555]:
#obtain the mean age to replace the missing values
train.mean(axis=0)

PassengerId    446.000000
Survived         0.383838
Pclass           2.308642
Age             29.699118
SibSp            0.523008
Parch            0.381594
Fare            32.204208
dtype: float64

In [0]:
#We are separating the data because we will compare the result of test_x with train_y
train_x = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived'],axis=1)
train_y = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'],axis=1)
test_x = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'],axis=1)

In [0]:
train_x['Age'] = train_x['Age'].fillna(30)
train_y['Age'] = train_y['Age'].fillna(30)

In [558]:
train_y.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,30.0,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [0]:
train_x["Embarked"].fillna("S", inplace = True) 
train_y["Embarked"].fillna("S", inplace = True) 

In [560]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [561]:
print(train_x.columns)

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')


In [562]:
print(test_x.columns)

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')


In [563]:
print(train_y.columns)

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked'],
      dtype='object')


**Input Function**

We need to create the input function which creates a `tf.data.Dataset` from a pandas dataframe

In [0]:
#Create training input function
train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=train_x,
                                                               y=train_y,
                                                               num_epochs=None,
                                                               batch_size=100,
                                                               shuffle=True
)
                                                      

**Feature Columns**

In this example, we have Categorical columns and numerical columns

In [0]:
feature_columns = []
num_cols = ['Age', 'Fare', 'Parch', 'Pclass', 'SibSp']
for num_name in num_cols:
  feature_columns.append(tf.feature_column.numeric_column(num))

categorical_cols = ['Sex', 'Embarked']
for ft_name in categorical_cols:
  vocabulary = train_x[ft_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(ft_name, vocabulary))

**Instantiate the Estimator**

We are going to use a Linear Classifier. This will train the model to classify the data into two possible cases: survival and not survival.

In [566]:
model = tf.estimator.LinearClassifier(feature_columns=feature_columns, optimizer='Adam')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpfpqq9svp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


**Train the model**

In [567]:
model.train(input_fn=train_input_fn, steps=10000)

INFO:tensorflow:Calling model_fn.


ValueError: ignored